In [ ]:
%matplotlib inline
import os
import sys
sys.path.append("..")
import time
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams['font.size'] = 10
rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Times New Roman']

from sklearn.decomposition import PCA

from canon.pattern.model import GMModel, KMeansModel, BGMModel, MeanShiftModel

def fill_features(features, img_shape):
    num_points = np.prod(img_shape)
    if len(features) != num_points:
        print("Filling %d features to %d data points" % (len(features), num_points))
        features2 = np.zeros((np.prod(img_shape), features.shape[1] - 1))
        features2[features[:, 0].astype('int') - 1] = features[:, 1:]
        return features2
    else:
        return features[:, 1:]

# Feature Extraction

In [ ]:
feature_file = "au29_area2_50_150_256"
img_shape = (50, 150)
aspect_ratio = 1.5

features = np.load("features/" + feature_file + ".npy")
features = fill_features(features, img_shape)

# Direct Coloring
Directly color each point using the first 3 principal components, without clustering.

In [ ]:
pca = PCA(n_components=3)
X = pca.fit_transform(features)
pca_range = [X.min(axis=0), X.max(axis=0)]
X = (X - pca_range[0]) / (pca_range[1] - pca_range[0])
Z = np.zeros((img_shape[0], img_shape[1], 3))
Z[:, :, 0] = X[:, 0].reshape(img_shape)
Z[:, :, 1] = X[:, 1].reshape(img_shape)
Z[:, :, 2] = X[:, 2].reshape(img_shape)
fig, ax = plt.subplots(ncols=1, nrows=1)
ax.imshow(Z[::-1, :, :], aspect=aspect_ratio)

# Clustering
The normal clustering + labeling

In [ ]:
model = KMeansModel()
pca = PCA(n_components=8)
model.train(features, n_clusters=11, preprocessors=[])

silhouette = model.compute_silhouette_score(features)
calinski = model.compute_calinski_harabaz_score(features)
print("Silhouette Score = {}, Calinski-Harabaz Score = {}".format(silhouette, calinski))

scores = np.array(model.score(features))

fig, ax = plt.subplots(ncols=1, nrows=1)

Z = model.color_by_pca(scores.reshape(img_shape), scaling='centroids')
ax.imshow(Z[::-1, :, :], aspect=aspect_ratio)
# fig.savefig("img/Z.pdf", bbox_inches='tight', figsize=(5, 4), dpi=300)

# Silhouette Score & Calinski-Harabaz Score
Use Silhouette Score & Calinski-Harabaz Score to estimate number of "coarse" clusters.
Good for grain boundaries and phase boundaries.

In [ ]:
from joblib import Parallel, delayed

Ks = np.arange(2, 20, 1)
SCs=[]

def inertia(k):
    print("K=", k)
    model = KMeansModel()
    pca = PCA(n_components=32)
    model.train(features, n_clusters=k, preprocessors=[pca])
    silhouette = model.compute_silhouette_score(features)
    calinski = model.compute_calinski_harabaz_score(features)
    print("Silhouette Score = {}, Calinski-Harabaz Score = {}".format(silhouette, calinski))
    return silhouette, calinski

with Parallel(n_jobs=-1, verbose=1) as parallel:
    SCs = parallel(delayed(inertia)(k) for k in Ks)
    SCs = np.array(SCs)
    
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].plot(Ks, SCs[:, 0], 'o--k', markerfacecolor='w', markersize=5)
ax[0].set_xlabel(r'Number of Clusters', fontsize=12)
ax[0].set_ylabel(r'Silhouette Score', fontsize=12)

ax[1].plot(Ks, SCs[:, 1], 'o--k', markerfacecolor='w', markersize=5)
ax[1].set_xlabel(r'Number of Clusters', fontsize=12)
ax[1].set_ylabel(r'Calinski-Harabaz Score', fontsize=12)

plt.tight_layout()